In [1]:
import pandas as pd
import json
import ast
import os
from pathlib import Path


def load_json_file(file_path, lines=False):
    path = Path(file_path)
    if not path.exists():
        print(f" Error: File not found -> {path}")
        return None
    try:
        df = pd.read_json(str(path), lines=lines)
        print(f" Loaded '{path.name}' with {len(df)} records.")
        return df
    except Exception as e:
        print(f" Error while reading {path}: {e}")
        return None

def load_all_json_from_folder(folder_path):
    folder = Path(folder_path)
    if not folder.exists():
        print(f" Error: Folder not found -> {folder}")
        return None

    all_dfs = []
    for file in folder.glob("*.json*"):  # matches .json and .jsonl
        is_jsonl = file.suffix == ".jsonl"
        df = load_json_file(file, lines=is_jsonl)
        if df is not None:
            all_dfs.append(df)

    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
        print(f"\n Combined {len(all_dfs)} files with {len(final_df)} records.")
        return final_df
    else:
        print(" No valid JSON files found in the folder.")
        return None

# === Load your folder ===
df = load_all_json_from_folder(
    r"C:\Users\rahusingh\OneDrive - Chegg Inc\Desktop\Sample_events_datasets"
)

if df is None:
    raise ValueError("No data loaded. Check your folder path.")


def safe_parse(row):
    try:
        if isinstance(row, str):
            return ast.literal_eval(row)  # safely parse Python-style strings
        elif isinstance(row, (list, dict)):
            return row
    except Exception:
        return None

parsed_data = df['data'].apply(safe_parse)

records = []
for item in parsed_data.dropna():
    if isinstance(item, list):
        records.extend(item)

events_df = pd.DataFrame(records)
print(" Flattened events:", events_df.shape)

# Expand attributes
if "attributes" in events_df.columns:
    attr_df = pd.json_normalize(events_df['attributes'])
    events_flat = pd.concat([events_df.drop(columns=['attributes']), attr_df], axis=1)
else:
    events_flat = events_df.copy()

# Extract company relationships
def extract_company_ids(rel):
    try:
        comp1 = rel.get("company1", {}).get("data", {}).get("id")
        comp2 = rel.get("company2", {}).get("data", {}).get("id")
        return pd.Series([comp1, comp2])
    except:
        return pd.Series([None, None])

if "relationships" in events_flat.columns:
    events_flat[["company1_id", "company2_id"]] = events_flat["relationships"].apply(extract_company_ids)
    events_flat = events_flat.drop(columns=["relationships"], errors="ignore")


included_records = []
for row in df["included"]:
    try:
        items = json.loads(row.replace("'", "\"")) if isinstance(row, str) else row
        included_records.extend(items)
    except:
        pass

companies_flat = pd.json_normalize(included_records)
companies_only = companies_flat[companies_flat["type"] == "company"].copy()

companies_clean = companies_only[[
    "id", "attributes.company_name", "attributes.domain", "attributes.ticker"
]].rename(columns={
    "id": "company_id",
    "attributes.company_name": "company_name",
    "attributes.domain": "domain",
    "attributes.ticker": "ticker"
}).drop_duplicates(subset=["company_id"])

print(" Cleaned companies:", companies_clean.shape)


events_clean = events_flat[[
    "id", "type", "summary", "category", "found_at", "confidence",
    "company1_id", "company2_id"
]].rename(columns={
    "id": "event_id",
    "type": "event_type",
    "found_at": "event_date"
})

print(" Cleaned events:", events_clean.shape)


def chunked_merge(events_df, companies_df, left_key, suffix, chunk_size=50000):
    chunks = []
    for i in range(0, len(events_df), chunk_size):
        chunk = events_df.iloc[i:i+chunk_size]
        merged = chunk.merge(
            companies_df,
            how="left",
            left_on=left_key,
            right_on="company_id"
        ).rename(columns={
            "company_name": f"{suffix}_name",
            "domain": f"{suffix}_domain",
            "ticker": f"{suffix}_ticker"
        }).drop(columns=["company_id"])
        chunks.append(merged)
        print(f" Processed {i:,} – {i+len(chunk):,} rows for {suffix}")
    return pd.concat(chunks, ignore_index=True)

events_with_company1 = chunked_merge(events_clean, companies_clean, "company1_id", "company1")
events_with_companies = chunked_merge(events_with_company1, companies_clean, "company2_id", "company2")

final_df = events_with_companies
print(" Final dataset:", final_df.shape)


output_csv = r"C:\Users\rahusingh\OneDrive - Chegg Inc\Desktop\Sample_events_datasets\final_dataset.csv"
output_parquet = r"C:\Users\rahusingh\OneDrive - Chegg Inc\Desktop\Sample_events_datasets\final_dataset.parquet"

final_df.to_csv(output_csv, index=False, encoding="utf-8")
final_df.to_parquet(output_parquet, index=False)

print(f"💾 Saved CSV -> {output_csv}")
print(f"💾 Saved Parquet -> {output_parquet}")


output_dir = r"C:\Users\rahusingh\OneDrive - Chegg Inc\Desktop\Sample_events_datasets\split_files"
os.makedirs(output_dir, exist_ok=True)

chunk_size = 100000  # adjust rows per chunk
num_chunks = (len(final_df) // chunk_size) + 1

for i in range(num_chunks):
    start = i * chunk_size
    end = start + chunk_size
    chunk = final_df.iloc[start:end]
    if not chunk.empty:
        out_path = os.path.join(output_dir, f"final_dataset_part{i+1}.csv")
        chunk.to_csv(out_path, index=False)
        print(f" Saved {out_path} with {len(chunk)} rows")

print("Splitting complete. Data ready for SQL.")

✅ Loaded 'news_events_2025_07_07.00000.jsonl' with 25422 records.
✅ Loaded 'news_events_2025_07_07.00001.jsonl' with 25934 records.
✅ Loaded 'news_events_2025_07_07.00002.jsonl' with 25442 records.
✅ Loaded 'news_events_2025_07_07.00003.jsonl' with 25412 records.
✅ Loaded 'news_events_2025_07_07.00004.jsonl' with 25985 records.
✅ Loaded 'news_events_2025_07_07.00005.jsonl' with 26205 records.
✅ Loaded 'news_events_2025_07_07.00050.jsonl' with 25549 records.
✅ Loaded 'news_events_2025_07_07.00052.jsonl' with 26851 records.
✅ Loaded 'news_events_2025_07_07.00053.jsonl' with 25545 records.
✅ Loaded 'news_events_2025_07_07.00058.jsonl' with 25427 records.
✅ Loaded 'news_events_2025_07_07.00076.jsonl' with 26000 records.
✅ Loaded 'news_events_2025_07_07.00077.jsonl' with 26559 records.
✅ Loaded 'news_events_2025_07_07.00078.jsonl' with 25867 records.
✅ Loaded 'news_events_2025_07_07.00080.jsonl' with 25745 records.
✅ Loaded 'news_events_2025_07_07.00083.jsonl' with 25735 records.
✅ Loaded '